In [1]:
import hyperopt
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import pandas as pd
import sklearn
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [73]:
train = pd.read_csv('credit_train.csv', sep=";",encoding='cp1251')
test = pd.read_csv('credit_test.csv', sep=";",encoding='cp1251')
le = preprocessing.LabelEncoder()
train['gender']=le.fit_transform(train['gender'])
train['marital_status']=le.fit_transform(train['marital_status'])
train['job_position']=le.fit_transform(train['job_position'])
train['education']=le.fit_transform(train['education'])
train['tariff_id']=le.fit_transform(train['tariff_id'])

list1 = list(train['living_region'].unique())

town_encoded = dict(zip(list1, range(len(list1))))

town_encoded

df = pd.DataFrame.from_dict(town_encoded,orient='index')

df = df.rename(columns={0:"town"})

df = df.reset_index()

df['living_region']=df['index']

df = df.drop('index',axis=1)


fin_df = pd.merge(train,df, on='living_region')

fin_df = fin_df.drop('living_region',axis=1)

fin_df = fin_df.set_index('client_id')

fin_df['credit_sum']=fin_df['credit_sum'].apply(lambda x: float(x.replace(',','.')))

fin_df['score_shk']=fin_df['score_shk'].apply(lambda x: float(x.replace(',','.')))
fin_df_train = fin_df

train = test
le = preprocessing.LabelEncoder()
train['gender']=le.fit_transform(train['gender'])

train['marital_status']=le.fit_transform(train['marital_status'])
train['job_position']=le.fit_transform(train['job_position'])
train['education']=le.fit_transform(train['education'])
train['tariff_id']=le.fit_transform(train['tariff_id'])

list1 = list(train['living_region'].unique())

town_encoded = dict(zip(list1, range(len(list1))))

town_encoded

df = pd.DataFrame.from_dict(town_encoded,orient='index')

df = df.rename(columns={0:"town"})

df = df.reset_index()

df['living_region']=df['index']

df = df.drop('index',axis=1)


fin_df = pd.merge(train,df, on='living_region')

fin_df = fin_df.drop('living_region',axis=1)

fin_df = fin_df.set_index('client_id')

fin_df['credit_sum']=fin_df['credit_sum'].apply(lambda x: float(x.replace(',','.')))

fin_df['score_shk']=fin_df['score_shk'].apply(lambda x: float(x.replace(',','.')))

fin_df_test = fin_df

#### Добавим новые, хорошо работающие фичи

In [74]:
fin_df_test['money_while_credit']= fin_df_test['credit_month']*fin_df_test['monthly_income']
fin_df_train['money_while_credit']= fin_df_train['credit_month']*fin_df_train['monthly_income']

In [75]:
y_train = fin_df_train['open_account_flg']
X_train = fin_df_train.drop('open_account_flg',axis = 1)
X_test = fin_df_test

#### Файл с блендингом разнородных моделей я не нашел, поэтому здесь представлен ансамбль из 3 xgboost-ов, который тоже отработал хорошо и вошел в финальный ансамбль

In [83]:
import numpy as np
np.random.seed(1301) # seed to shuffle the train set

n_folds = 5
verbose = True
shuffle = False

training = fin_df_train
test = fin_df_test
print(training.shape)
print(test.shape)


X = fin_df_train.drop('open_account_flg',axis = 1)
y = fin_df_train['open_account_flg']

# Feature selection with xgboost feature selection


X_sel = fin_df_train.drop('open_account_flg',axis = 1)
sel_test = fin_df_test
X, y, X_submission = np.array(X_sel), np.array(y), np.array(sel_test)

if shuffle:
    idx = np.random.permutation(y.size)
    X = X[idx]
    y = y[idx]
#ratio = float(np.sum(y == 1)) / np.sum(y==0)
clfs = [xgb.XGBClassifier(#base_score = 0.3,
                     objective = "binary:logistic", 
                     max_depth = 18,
                     n_estimators = 3021,
                     learning_rate = 0.02, 
                     subsample = 0.85,
                     colsample_bytree = 0.24,
                     silent = 1,
                     min_child_weight = 2,
                     nthread = 5,
                     gamma = 1.4,
                     seed = 1301),
        xgb.XGBClassifier(#base_score = 0.3,
                     objective = "binary:logistic", 
                     max_depth = 7,
                     n_estimators = 8511,
                     learning_rate = 0.01, 
                     subsample = 0.6,
                     colsample_bytree = 0.38,
                     silent = 1,
                     min_child_weight = 1,
                     nthread = 5,
                     gamma = 0.9,
                     seed = 2017),
        xgb.XGBClassifier(#base_score = 0.3,
                     objective = "binary:logistic", 
                     max_depth = 15,
                     n_estimators = 3021,
                     learning_rate = 0.02, 
                     subsample = 0.85,
                     colsample_bytree = 0.41,
                     silent = 1,
                     min_child_weight = 5,
                     nthread = 5,
                     gamma = 1.4,
                     seed = 1347)]



(170746, 15)
(91940, 14)


In [84]:
print ("Creating train and test sets for blending.")
        
dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
dataset_blend_test = np.zeros((X_submission.shape[0], len(clfs)))
skf = cross_validation.StratifiedKFold(y, n_folds, shuffle=True)

Creating train and test sets for blending.


In [85]:
for j, clf in enumerate(clfs):
    print (j, clf)
    dataset_blend_test_j = np.zeros((X_submission.shape[0], n_folds))
    for i, (train, testidx) in enumerate(skf):
        print ("Fold", i)
        X_train, y_train = X[train], y[train]
        X_test, y_test = X[testidx], y[testidx]
        clf.fit(X_train, y_train)
        y_submission = clf.predict_proba(X_test)[:,1]
        dataset_blend_train[testidx, j] = y_submission
        dataset_blend_test_j[:, i] = clf.predict_proba(X_submission)[:,1]
    dataset_blend_test[:,j] = dataset_blend_test_j.mean(axis=1)

0 XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.24,
       gamma=1.4, learning_rate=0.02, max_delta_step=0, max_depth=18,
       min_child_weight=2, missing=None, n_estimators=3021, nthread=5,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=1301, silent=1, subsample=0.85)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
1 XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.38,
       gamma=0.9, learning_rate=0.01, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=8511, nthread=5,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=2017, silent=1, subsample=0.6)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
2 XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.41,
       gamma=1.4, learning_rate=0.02, max_delta_step=0, max_depth=15,
       min_child_weight=5, missing=None, n_estimators=3021, nthread=5,
       objective='binary:lo

#### Сделаем блендинг и сохраним итоговый файл для сабмита

In [86]:
print ("Blending.")
clf = LogisticRegression(C = 0.6, intercept_scaling = 0.5)
clf.fit(dataset_blend_train, y)
y_training = clf.predict_proba(dataset_blend_train)[:,1]
y_submission = clf.predict_proba(dataset_blend_test)[:,1]
    
print ("Linear stretch of predictions to [0,1]")
y_training = (y_training - y_training.min()) / (y_training.max() - y_training.min())
y_submission = (y_submission - y_submission.min()) / (y_submission.max() - y_submission.min())
    
print ("Saving Results.")
submission = pd.DataFrame({"_ID_":test.index, "_VAL_":y_submission})
submission.to_csv("submission_GBT_XGB.csv", index=False)

from sklearn.metrics import roc_auc_score

auc_score = roc_auc_score(y, y_training)
print ("AUC=", auc_score)

Blending.
Linear stretch of predictions to [0,1]
Saving Results.
AUC= 0.765513500063
